In [1]:
from data_provider.data_factory import data_provider
from utils.tools import EarlyStopping, adjust_learning_rate, visual, vali, test
from torch.utils.data import Subset
from tqdm import tqdm
from models.PatchTST import PatchTST
from models.GPT4TS import GPT4TS
from models.DLinear import DLinear
from models.TEMPO import TEMPO
# from models.T5 import T54TS
from models.ETSformer import ETSformer


import numpy as np
import torch

In [2]:
import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

import torch.nn as nn
from torch import optim
from numpy.random import choice

import os
import time

import warnings
import matplotlib.pyplot as plt
import numpy as np

import argparse
import random
import sys

from omegaconf import OmegaConf

In [3]:
def get_init_config(config_path=None):
    config = OmegaConf.load(config_path)
    return config

In [4]:
from omegaconf import OmegaConf

# Define the configuration dictionary
config = {
    "datasets": {
        "Traffic": {
            "root_path": "/u/dcao1/workspace/Generative-TS-train_all/csdi_data/traffic_nips/",
            "data_path": "train",
            "data_name": "Traffic",
            "data": "traffic",
            "lradj": "type4",
            "features": "M",
            "target": "400001",
            "embed": "timeF",
            "freq": 0,
            "percent": 100
        }
    }
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)


In [5]:

# Save the configuration to a YAML file
with open("./configs/custom_dataset.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration saved to ./configs/custom_dataset.yml")

Configuration saved to ./configs/custom_dataset.yml


In [6]:
config = {
    "description": "TEMPO",
    # "checkpoints": "checkpoints/lora_revin_6domain_checkpoints_1/",
    "checkpoints": "checkpoints/lora_revin_5domain_checkpoints_1/",
    "task_name": "long_term_forecast",
    "prompt": 1,
    "num_nodes": 1,
    "seq_len": 168,
    "pred_len": 30,
    "label_len": 30,
    "decay_fac": 0.5,
    "learning_rate": 0.001,
    "batch_size": 256,
    "num_workers": 0,
    "train_epochs": 10,
    "lradj": "type3",
    "patience": 5,
    "gpt_layers": 6,
    "is_gpt": 1,
    "e_layers": 3,
    "d_model": 768,
    "n_heads": 4,
    "d_ff": 768,
    "dropout": 0.3,
    "enc_in": 7,
    "c_out": 1,
    "patch_size": 16,
    "kernel_size": 25,
    "loss_func": "mse",
    "pretrain": 1,
    "freeze": 1,
    "model": "TEMPO",
    "stride": 8,
    "max_len": -1,
    "hid_dim": 16,
    "tmax": 20,
    "itr": 3,
    "cos": 1,
    "equal": 1,
    "pool": False,
    "no_stl_loss": False,
    "stl_weight": 0.001,
    "config_path": "./configs/custom_dataset.yml",
    "datasets": "ETTm1,ETTh1,ETTm2,electricity,traffic,weather",
    "target_data": "Traffic",
    "use_token": 0,
    "electri_multiplier": 1,
    "traffic_multiplier": 1,
    "embed": "timeF",
    "percent": 100,
    # "model_id": 'prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll168_pl24_dm768_nh4_el3_gl6_df768_ebtimeF_itr0' #"prob_TEMPO_6_prompt_learn_168_24_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
     "model_id": "prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0"
}

# Create an OmegaConf object from the dictionary
cfg = OmegaConf.create(config)

# Save the configuration to a YAML file
with open("configs/etth2_config.yml", "w") as f:
    OmegaConf.save(cfg, f)

print("Configuration written to config.yml")

Configuration written to config.yml


In [7]:
config = get_init_config(cfg.config_path)

In [8]:
SEASONALITY_MAP = {
   "minutely": 1440,
   "10_minutes": 144,
   "half_hourly": 48,
   "hourly": 24,
   "daily": 7,
   "weekly": 1,
   "monthly": 12,
   "quarterly": 4,
   "yearly": 1
}

In [9]:
device = torch.device('cuda:0')

In [10]:
if cfg.model == 'PatchTST':
    model = PatchTST(cfg, device)
    model.to(device)
elif cfg.model == 'DLinear':
    model = DLinear(cfg, device)
    model.to(device)
elif cfg.model == 'TEMPO':
    model = TEMPO(cfg, device)
    model.to(device)
elif cfg.model == 'T5':
    model = T54TS(cfg, device)
    model.to(device)
elif 'ETSformer' in cfg.model:
    model = ETSformer(cfg, device)
    model.to(device)
else:
    model = GPT4TS(cfg, device)


params = model.parameters()
model_optim = torch.optim.Adam(params, lr=cfg.learning_rate)

early_stopping = EarlyStopping(patience=cfg.patience, verbose=True)
if cfg.loss_func == 'mse':
    criterion = nn.MSELoss()
elif cfg.loss_func == 'smape':
    class SMAPE(nn.Module):
        def __init__(self):
            super(SMAPE, self).__init__()
        def forward(self, pred, true):
            return torch.mean(200 * torch.abs(pred - true) / (torch.abs(pred) + torch.abs(true) + 1e-8))
    criterion = SMAPE()

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=cfg.tmax, eta_min=1e-8)

trainable params: 308736 || all params: 82207488 || trainable%: 0.38


In [11]:
model_path = os.path.join(cfg.checkpoints, cfg.model_id)
best_model_path = model_path + '/checkpoint.pth'
print(best_model_path)
model.load_state_dict(torch.load(best_model_path,  map_location=torch.device(device)), strict=False)
print("------------------------------------")

checkpoints/lora_revin_5domain_checkpoints_1/prob_TEMPO_6_prompt_learn_168_30_100_sl336_ll30_pl30_dm768_nh4_el3_gl6_df768_ebtimeF_itr0/checkpoint.pth
------------------------------------


In [12]:
cfg.data = config['datasets'][cfg.target_data].data
cfg.root_path = config['datasets'][cfg.target_data].root_path
cfg.data_path = config['datasets'][cfg.target_data].data_path
cfg.data_name = config['datasets'][cfg.target_data].data_name
cfg.features = config['datasets'][cfg.target_data].features
cfg.freq = config['datasets'][cfg.target_data].freq
cfg.target = config['datasets'][cfg.target_data].target
cfg.embed = config['datasets'][cfg.target_data].embed
cfg.percent = config['datasets'][cfg.target_data].percent
cfg.lradj = config['datasets'][cfg.target_data].lradj
if cfg.freq == 0:
    cfg.freq = 'h'
cfg.seq_len = 168 + 24
cfg.pred_len = 24 
test_data, test_loader = data_provider(cfg, 'test')

self.enc_in = 963
self.data_x = (7, 963)
test 7


In [13]:
test_data.mean_data

0

In [14]:
# %%
import torch.distributions as dist
preds = []
trues = []
history = []
# mases = []

# Initialize accumulators for errors
total_mae = 0
total_mse = 0
n_samples = 0
itr = 0

cfg.seq_len = 168
cfg.pred_len = 24
model.eval()
# import pdb; pdb.set_trace()
with torch.no_grad():
    for i in range(len(test_data)):
        #enumerate(test_loader): #tqdm(enumerate(test_loader), total=len(test_loader)):
        data = test_data[i]
        seq_x, text_embedding, observed_mask = data[0].to(device), data[1].to(device), data[2].to(device)
        seq_x = seq_x.float().to(device)
        batch_x = seq_x[:, :cfg.seq_len].unsqueeze(-1) # 137, 168, 1
        batch_y = seq_x[:,cfg.seq_len:cfg.seq_len+cfg.pred_len]
        trues.append(batch_y.cpu().numpy())
        history.append(batch_x.cpu().numpy())
                        
        outputs, _ = model(batch_x, 0, None, None, None) #+ model(seq_seasonal, ii) + model(seq_resid, ii)

        mu, sigma, nu = outputs[0], outputs[1], outputs[2]
        # Create the Student's t-distribution with the predicted parameters
        student_t = dist.StudentT(df=nu, loc=mu, scale=sigma)

        # Generate 30 samples for each prediction
        num_samples = 35
        probabilistic_forecasts = student_t.rsample((num_samples,))

        # The shape of probabilistic_forecasts will be (num_samples, batch_size, pred_length)
        print(probabilistic_forecasts.shape)
        preds.append(probabilistic_forecasts.cpu().numpy())


torch.Size([35, 963, 30])
torch.Size([35, 963, 30])
torch.Size([35, 963, 30])
torch.Size([35, 963, 30])
torch.Size([35, 963, 30])
torch.Size([35, 963, 30])
torch.Size([35, 963, 30])


In [15]:
trues[0]

array([[0.03081667, 0.02231667, 0.01846667, ..., 0.02955   , 0.02948333,
        0.02785   ],
       [0.12431667, 0.12183333, 0.11065   , ..., 0.11313333, 0.11663333,
        0.11951666],
       [0.06731667, 0.05773333, 0.05038333, ..., 0.06418333, 0.06238334,
        0.0611    ],
       ...,
       [0.05115   , 0.05063333, 0.04966667, ..., 0.05776667, 0.0551    ,
        0.05406667],
       [0.0856    , 0.07065   , 0.05335   , ..., 0.07421666, 0.07233334,
        0.07168333],
       [0.1395    , 0.0727    , 0.06136667, ..., 0.06471667, 0.08513334,
        0.0609    ]], dtype=float32)

In [16]:
preds[0]

array([[[ 1.24144331e-02,  6.81488514e-02,  2.82850452e-02, ...,
          2.01824252e-02, -3.94613743e-02,  8.15666094e-03],
        [ 9.81072262e-02,  9.96413007e-02,  8.41377825e-02, ...,
          1.40915245e-01,  7.33797550e-02,  4.46842641e-01],
        [ 5.91544844e-02,  6.04421571e-02,  7.24687278e-02, ...,
          3.56234908e-02, -4.55199629e-02, -1.52937323e-02],
        ...,
        [ 2.98822075e-02,  4.35462743e-02,  4.07800041e-02, ...,
         -7.40120709e-02,  1.99849471e-01,  1.12962112e-01],
        [ 7.85163939e-02,  1.25706121e-02, -5.87195903e-03, ...,
          1.45974725e-01,  2.58007556e-01,  7.21831024e-02],
        [ 1.45941585e-01,  1.60609275e-01,  9.61162150e-02, ...,
          1.06624857e-01,  5.00327162e-03,  3.87874097e-02]],

       [[ 8.28624703e-03,  8.74084793e-03,  2.00791787e-02, ...,
          2.80775093e-02, -6.82478920e-02, -3.59249301e-04],
        [ 1.20261207e-01,  1.09147258e-01,  8.88639241e-02, ...,
          1.12656891e-01,  6.91867322e

In [17]:
np.array(preds).shape, np.array(trues).shape

((7, 35, 963, 30), (7, 963, 24))

In [18]:
gt_data = np.array(trues)
synthetic_data = np.array(preds)
history_data = np.array(history)

In [19]:
synthetic_data = synthetic_data[:,:,:, :gt_data.shape[2]]
target_mask = np.ones_like(gt_data)

In [20]:
# synthetic_data = np.swapaxes(synthetic_data, -1, -2)
# gt_data = np.swapaxes(gt_data, -1, -2)
# synthetic_data.shape

In [21]:
from metrics.imputation_metrics import mse_withmask, mae_withmask, calc_quantile_CRPS, calc_quantile_CRPS_sum

In [22]:
calc_quantile_CRPS_sum(torch.Tensor(gt_data),torch.Tensor(synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

0.14691095603139778

In [171]:
# synthetic_data = np.array(synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
# gt_data = np.array(gt_data).reshape(gt_data.shape[0],gt_data.shape[2], -1)
# target_mask = np.ones_like(gt_data)

In [189]:
unormzalized_his_data = []
for g in history_data:
    unormzalized_his_data.append(np.transpose(np.transpose(g)*test_data.std_data+test_data.mean_data))
unormzalized_his_data = np.array(unormzalized_his_data)

In [172]:
unormzalized_gt_data = []
for g in gt_data:
    unormzalized_gt_data.append(np.transpose(np.transpose(g)*test_data.std_data+test_data.mean_data))
unormzalized_gt_data = np.array(unormzalized_gt_data)
unormalized_synthetic_data = []
for i in range(len(synthetic_data)):
    for j in range(len(synthetic_data[i])):
        s = synthetic_data[i][j]
        unormalized_synthetic_data.append(np.transpose(np.transpose(s)*test_data.std_data+test_data.mean_data))
unormalized_synthetic_data = np.array(unormalized_synthetic_data).reshape(synthetic_data.shape[0],-1,synthetic_data.shape[2],synthetic_data.shape[3])
unormzalized_gt_data.shape,unormalized_synthetic_data.shape

((7, 963, 24), (7, 35, 963, 24))

In [173]:
unormzalized_gt_data.shape

(7, 963, 24)

In [174]:
unormalized_synthetic_data.shape

(7, 35, 963, 24)

In [175]:
gt_data.shape

(7, 963, 24)

In [176]:
unormalized_synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)
unormzalized_gt_data = np.swapaxes(unormzalized_gt_data, -1, -2)


In [177]:
target_mask = np.ones_like(unormzalized_gt_data)

In [178]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.03227819580780832

In [179]:
calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1)


0.03227819580780832

In [147]:
print("all:   CRPS_sum:")
print(calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data),torch.Tensor(target_mask),mean_scaler=0,scaler=1))
print("all: CRPS_sum:")
# import pdb; pdb.set_trace()
CRPS_sum_1 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,:10]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_2 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,10:20]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)
CRPS_sum_3 = calc_quantile_CRPS_sum(torch.Tensor(unormzalized_gt_data),torch.Tensor(unormalized_synthetic_data[:,20:30]),torch.Tensor(target_mask),mean_scaler=0,scaler=1)

CRPS_sum_mean = np.mean([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
CRPS_sum_std = np.std([CRPS_sum_1, CRPS_sum_2, CRPS_sum_3])
print("Mean CRPS_sum:", CRPS_sum_mean)
print("Standard Deviation CRPS_sum:", CRPS_sum_std)

all:   CRPS_sum:
0.024586759115520277
all: CRPS_sum:
Mean CRPS_sum: 0.025835803203415452
Standard Deviation CRPS_sum: 0.00038745621392132987


In [186]:
low_q = np.quantile(synthetic_data,0.05,axis=1)
high_q = np.quantile(synthetic_data,0.95,axis=1)
mid_q = np.quantile(synthetic_data,0.5,axis=1)



In [193]:
history_data.shape, gt_data.shape

((7, 963, 168, 1), (7, 963, 24))

In [195]:
traj_data = np.concatenate([history_data.squeeze(), gt_data], axis=2)
traj_data.shape

(7, 963, 192)

In [207]:
import pandas as pd
import matplotlib.pyplot as plt
i = 0
#3000
seq_len = 192
feat_dim = synthetic_data.shape[2]
feat_dim = 10
fig, axes = plt.subplots(nrows=feat_dim, ncols=1, figsize=(10, 20))
for feat_idx in range(feat_dim):
    # df_x = pd.DataFrame({"x": np.arange(0, seq_len), "val": gt_data[i, feat_idx, :],
    #                      "y": (1-target_mask)[i, feat_idx, :]})
    # df_x = df_x[df_x.y!=0]
    print(feat_idx, low_q.shape)
    df_o = pd.DataFrame({"x": np.arange(0, seq_len), "val": traj_data[i, feat_idx, :]})
                        # "y": target_mask[i, feat_idx, :]})
    # df_o = df_o[df_o.y!=0]
    axes[feat_idx].plot(df_o.x, df_o.val, color='b',  linestyle='solid', label='label')
    # axes[feat_idx].plot(df_x.x, df_x.val, color='r', marker='x', linestyle='None')
    axes[feat_idx].plot(range(seq_len-24, seq_len), mid_q[i, feat_idx, -24:], color='g', linestyle='solid', label='Diffusion-TS')
    axes[feat_idx].fill_between(range(seq_len-24, seq_len), low_q[i, feat_idx,-24:],high_q[i,feat_idx, -24:], color='g', alpha = 0.3)
    plt.setp(axes[feat_idx], ylabel='value')
    if feat_idx == feat_dim-1:
        plt.setp(axes[-1], xlabel='time')
    # plt.show()
    # axes[feat_idx].set_ylim(-3, 3)
plt.legend()
plt.savefig('traffic_1.png')
plt.show()

0 (7, 963, 24)
1 (7, 963, 24)
2 (7, 963, 24)
3 (7, 963, 24)
4 (7, 963, 24)
5 (7, 963, 24)
6 (7, 963, 24)
7 (7, 963, 24)
8 (7, 963, 24)
9 (7, 963, 24)


In [180]:
from metrics.WIS import score

In [181]:
synthetic_data = np.swapaxes(unormalized_synthetic_data, -1, -2)

labels = np.swapaxes(unormzalized_gt_data, -1, -2)

In [182]:
low_q = np.quantile(synthetic_data,0.05,axis=1)
high_q = np.quantile(synthetic_data,0.95,axis=1)
mid_q = np.quantile(synthetic_data,0.5,axis=1)

In [183]:
high_q_reshape = np.reshape(high_q,newshape=(-1,high_q.shape[-1]))
low_q_reshape = np.reshape(low_q,newshape=(-1,low_q.shape[-1]))
gt_data_reshape = np.reshape(labels,newshape=(-1,labels.shape[-1]))
print(high_q_reshape.shape,low_q_reshape.shape,gt_data_reshape.shape)

(6741, 24) (6741, 24) (6741, 24)


In [184]:
alpha = 0.05 # to be set by competition hosts
if not(alpha > 0 and alpha <= 1):
    raise ParticipantVisibleError(f'alpha should be (0,1]. Found {alpha}')

MWIS,coverage = score(gt_data_reshape,high_q_reshape,low_q_reshape,alpha)

if not(coverage >= (1-alpha)):
    raise ParticipantVisibleError(f'Coverage was below {(1-alpha)*100}%')

In [185]:
MWIS,coverage

(0.2063269790973157, 23.679424417742176)